In [1]:
from transformers import AutoProcessor
from modeling_git import GitForCausalLM, GitModel, GitForCausalLMClipEmb
from PIL import Image
import torch
import requests


/root/miniconda3/envs/mindeye/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [3]:
from transformers import AutoProcessor, AutoModelForCausalLM
from modeling_git import GitForCausalLMClipEmb
processor = AutoProcessor.from_pretrained("microsoft/git-large-coco")
clip_text_model = GitForCausalLMClipEmb.from_pretrained("microsoft/git-large-coco")
clip_text_model.to(device) # if you get OOM running this script, you can switch this to cpu and lower minibatch_size to 4
clip_text_model.eval().requires_grad_(False)

# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)

# git_image = Image.open("/root/autodl-tmp/Workspace/EEG_caption/docs/test/banana_09s.jpg")
# pixel_values = processor(images=git_image, return_tensors="pt").pixel_values.to(device)
# vision_encoder=model.git.image_encoder

# git_image_features=vision_encoder(pixel_values).last_hidden_state.cpu()
# git_image_features.shape


/root/miniconda3/envs/mindeye/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


GitForCausalLMClipEmb(
  (git): GitModelClipEmb(
    (embeddings): GitEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(1024, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (image_encoder): GitVisionModel(
      (vision_model): GitVisionTransformer(
        (embeddings): GitVisionEmbeddings(
          (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
          (position_embedding): Embedding(257, 1024)
        )
        (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (encoder): GitVisionEncoder(
          (layers): ModuleList(
            (0-23): 24 x GitVisionEncoderLayer(
              (self_attn): GitVisionAttention(
                (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (v_proj): Linear(in_features=1024, out_features=1024, bia

In [4]:
git_test = torch.load("/root/autodl-tmp/Workspace/EEG_caption/ViT-L-14_features_GIT_test.pt")['img_features']
git_test.shape

torch.Size([200, 257, 1024])

In [5]:
emb_eeg_test = torch.load('/root/autodl-tmp/Workspace/EEG_caption/ATM_S_eeg_features_sub-08_test.pt').unsqueeze(1)
emb_eeg_test.shape

torch.Size([200, 1, 1024])

In [6]:
# image_features.std()

In [7]:
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import open_clip
from matplotlib.font_manager import FontProperties

import sys
from diffusion_prior import *
from custom_pipeline import *
# os.environ["CUDA_VISIBLE_DEVICES"] = "5" 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from einops.layers.torch import Rearrange, Reduce

# Define the neural network
class PixelProjector(nn.Sequential):
    def __init__(self, proj_dim=1024):
        super().__init__(
            Rearrange('B C L->B L C'),    
            nn.Linear(1, 257),
            nn.LayerNorm(257),
            Rearrange('B L C->B C L'),
            nn.Linear(1024, 1024),
            nn.LayerNorm(proj_dim),
            )
model = PixelProjector(proj_dim=1024).to(torch.bfloat16).to(device)
model.load_state_dict(torch.load('/root/autodl-tmp/Workspace/EEG_caption/model_weights/PixelProjector_best.bin'))


<All keys matched successfully>

In [9]:
diffusion_prior = DiffusionPriorUNet(cond_dim=1024, dropout=0.1)
# number of parameters
print(sum(p.numel() for p in diffusion_prior.parameters() if p.requires_grad))
pipe = Pipe(diffusion_prior, device=device)

# load pretrained model
model_name = 'diffusion_prior' # 'diffusion_prior_vice_pre_imagenet' or 'diffusion_prior_vice_pre'
# pipe.train(dl, num_epochs=150, learning_rate=1e-3) # to 0.142 
pipe.diffusion_prior.load_state_dict(torch.load(f'{model_name}.pt', map_location=device))

9675648


<All keys matched successfully>

In [10]:
# Loop through each element in emb_eeg_test and generate captions
with open('semantic_level_caption.txt', 'w') as f:
    for emb in emb_eeg_test:
        # Generate h for each emb
        h = pipe.generate(c_embeds=emb, num_inference_steps=4, guidance_scale=5.0)

        # Get test image
        test_img_257_1024 = model(h.unsqueeze(0).to(torch.bfloat16).to(device))

        # Generate captions
        generated_ids = clip_text_model.generate(pixel_values=test_img_257_1024.to(device).float(), max_new_tokens=25)
        generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)
        print(generated_caption)
        # Write each caption to a new line in the txt file
        f.write(f"{generated_caption[0]}\n")

print("Captions generated and saved to 'generated_captions.txt'.")


4it [00:00, 11.67it/s]
4it [00:00, 353.27it/s]
4it [00:00, 365.84it/s]
4it [00:00, 374.32it/s]
4it [00:00, 364.18it/s]
4it [00:00, 369.63it/s]
4it [00:00, 366.92it/s]
4it [00:00, 366.76it/s]
4it [00:00, 361.30it/s]
4it [00:00, 367.80it/s]
4it [00:00, 364.50it/s]
4it [00:00, 345.36it/s]
4it [00:00, 360.85it/s]
4it [00:00, 359.27it/s]
4it [00:00, 367.69it/s]
4it [00:00, 359.10it/s]
4it [00:00, 360.75it/s]
4it [00:00, 363.47it/s]
4it [00:00, 361.98it/s]
4it [00:00, 357.24it/s]
4it [00:00, 363.58it/s]
4it [00:00, 361.63it/s]
4it [00:00, 359.38it/s]
4it [00:00, 360.46it/s]
4it [00:00, 355.67it/s]
4it [00:00, 356.56it/s]
4it [00:00, 358.51it/s]
4it [00:00, 360.47it/s]
4it [00:00, 359.57it/s]
4it [00:00, 358.81it/s]
4it [00:00, 360.69it/s]
4it [00:00, 360.58it/s]
4it [00:00, 357.33it/s]
4it [00:00, 358.23it/s]
4it [00:00, 359.55it/s]
4it [00:00, 360.18it/s]
4it [00:00, 352.28it/s]
4it [00:00, 356.47it/s]
4it [00:00, 289.61it/s]
4it [00:00, 344.86it/s]
4it [00:00, 248.81it/s]
4it [00:00, 249.0

Captions generated and saved to 'generated_captions.txt'.


In [ ]:
# dataset = EmbeddingDataset(
#     c_embeddings=eeg_features_train, h_embeddings=emb_img_train_4, 
#     # h_embeds_uncond=h_embeds_imgnet
# )
# dl = DataLoader(dataset, batch_size=1024, shuffle=True, num_workers=64)
diffusion_prior = DiffusionPriorUNet(cond_dim=1024, dropout=0.1)
# number of parameters
print(sum(p.numel() for p in diffusion_prior.parameters() if p.requires_grad))
pipe = Pipe(diffusion_prior, device=device)

# load pretrained model
model_name = 'diffusion_prior' # 'diffusion_prior_vice_pre_imagenet' or 'diffusion_prior_vice_pre'
# pipe.train(dl, num_epochs=150, learning_rate=1e-3) # to 0.142 
pipe.diffusion_prior.load_state_dict(torch.load(f'{model_name}.pt', map_location=device))

h = pipe.generate(c_embeds=emb_eeg_test[0], num_inference_steps=4, guidance_scale=5.0)


model = PixelProjector(proj_dim=1024).to(torch.bfloat16).to(device)
model.load_state_dict(torch.load('/root/autodl-tmp/Workspace/EEG_caption/model_weights/PixelProjector_best.bin'))


test_img_257_1024 = model(h.unsqueeze(0).to(torch.bfloat16).to(device))

generated_ids = clip_text_model.generate(pixel_values=test_img_257_1024.to(device).float(), max_new_tokens=25)

generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)
generated_caption

In [ ]:
# generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
# generated_caption